In [4]:
!pip3 install transformers

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import re
def get_data(filepath):
	vocab = set()
	genes = []
	labels = []
	max_len = 0
	count = 0
	with open(filepath, 'r') as f:
		l = f.readline()
		l = f.readline()
		while l != None:
			inside_bracks = re.findall(r'\[(.*?)\]',l)
			if len(inside_bracks) < 2:
				break
			if count == 300:
				break
			mers = inside_bracks[0].split(", ")
			mers = [mer[1:-1] for mer in mers]
			if len(mers) > max_len:
				max_len = len(mers)
			genes.append(mers)
			gene_labels = inside_bracks[1].split(", ")
			gene_labels = list(map(float, gene_labels)) 
			labels.append(gene_labels)
			vocab.update(mers)
			l = f.readline()
			count += 1
	return vocab, genes, labels, max_len

def pad(genes, max_len):
	atten_masks = []
	for i in range(len(genes)):
		length = len(genes[i])
		diff = max_len - length
		genes[i].extend(['[PAD]' for i in range(diff)])
		mask = [1 for j in range(length)]
		mask.extend([0 for j in range(diff)])
		atten_masks.append(np.array(mask))
	return genes, np.array(atten_masks)

In [0]:
import torch
from transformers import TransfoXLTokenizer, TransfoXLModel, TransfoXLConfig
class Model(torch.nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.config = TransfoXLConfig(vocab_size_or_config_json_file=len(vocab)+267735)
    self.model = TransfoXLModel(self.config)
    self.tokenizer = TransfoXLTokenizer.from_pretrained('transfo-xl-wt103')
    self.out_layer = torch.nn.Linear(self.model.d_model, 2)
  def forward(self, input_ids, mems=None):
    hidden_states, mems = self.model(input_ids, mems)
    preds = self.out_layer(hidden_states).squeeze(0)
    return preds, mems

In [0]:
from torch.nn import CrossEntropyLoss
import numpy as np
batch_size = 1
window_size = 1012
vocab, genes, labels, max_len = get_data("/content/drive/My Drive/all_samples_6-mer.txt")
#genes, atten_masks = pad(genes, max_len)
model = Model()
model.tokenizer.add_tokens(list(vocab))
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
class_weights = torch.tensor(np.array([1.0, 100.0])).float()
loss = CrossEntropyLoss(weight=class_weights)
total_loss = 0
for i in range(0, len(genes)):
	mems = None
	print("running on gene: " + str(i))
	gene_loss = 0
	for w in range(0, len(genes[i]), window_size):
		toks = np.array(model.tokenizer.convert_tokens_to_ids(genes[i][w:w+window_size]))
		input_ids = torch.tensor(toks).unsqueeze(0)
		gene_labels = torch.tensor(np.array(labels[i][w:w+window_size])).long()
		predictions, mems = model(input_ids, mems)
		optimizer.zero_grad()
		l = loss(predictions, gene_labels)
		gene_loss += l.item()
		l.backward()
		optimizer.step()
	total_loss += gene_loss
	if i > 0 and i % 10 == 0:
		print("Epoch: " + str(i) + " loss: " + str(total_loss / 10))
		total_loss = 0


running on gene: 0
running on gene: 1
running on gene: 2
running on gene: 3
running on gene: 4
running on gene: 5
running on gene: 6
running on gene: 7
running on gene: 8
running on gene: 9
running on gene: 10
Epoch: 10 loss: 4.566710933297804
running on gene: 11
running on gene: 12
running on gene: 13
running on gene: 14
running on gene: 15
running on gene: 16
running on gene: 17
running on gene: 18
running on gene: 19
running on gene: 20
Epoch: 20 loss: 1.8461555227637292
running on gene: 21
running on gene: 22
running on gene: 23
running on gene: 24
running on gene: 25
running on gene: 26
running on gene: 27
running on gene: 28
running on gene: 29
running on gene: 30
Epoch: 30 loss: 1.2210110008716584
running on gene: 31
running on gene: 32
running on gene: 33
running on gene: 34
running on gene: 35
running on gene: 36
running on gene: 37
running on gene: 38
running on gene: 39
running on gene: 40
Epoch: 40 loss: 3.9490367114543914
running on gene: 41
